In [ ]:
import torch
import os
import sys
sys.path.append('./../../src')
import random
import numpy as np

In [ ]:
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
random.seed()
np.random.seed(0)

In [ ]:
dev_data_path = './sanity-checks/esnli_dev_100.csv'
save_trained_model_dir = './esnli_train_trained_model/'

In [ ]:
from transformers import BertTokenizer, EncoderDecoderModel
import csv
from esnli_processor import *
import logging as logger

In [ ]:
output_dir = save_trained_model_dir
cuda_id = "1" # since there's something running on 0

In [ ]:
processor = EsnliProcessor()
dev_examples = processor.get_dev_examples(dev_data_path) 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
max_seq_len = 128
dev_features = esnli_examples_to_features(dev_examples, max_seq_len, tokenizer)

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
model = EncoderDecoderModel.from_pretrained(output_dir)
device = torch.device("cuda:"+cuda_id)
model.to(device)

### New evaluation:

In [ ]:
model.config.max_length=100

In [ ]:
outputs = model.generate(input_ids=torch.tensor([dev_features[0].input_ids, 
                                       dev_features[1].input_ids]).to(device), 
               attention_mask=torch.tensor([dev_features[0].attention_mask, 
                                            dev_features[1].attention_mask]).to(device),
               decoder_start_token_id=101)

In [ ]:
outputs

In [ ]:
output_txt1 = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
output_txt1

In [ ]:
output_txt2 = tokenizer.batch_decode(outputs)

In [ ]:
output_txt2

In [ ]:
tokenizer.decode